### University of Toronto, Department of Electrical and Computer Engineering

## ECE 1501 &mdash; Error Control Codes

# Exercise 2: Reed&ndash;Solomon Codes

### Last name: 
### First name: 
### Student number:

The purpose of this Numerical Exercise is to implement Reed&ndash;Solomon codes and some of their subfield subcodes. There are five parts to this numerical exercise:

0. **Polynomials:**  You will learn how to work with polynomials over finite fields in julia. This exercise will not be graded, but it will be important to solve as it will be needed for following exercises!

1. **Reed&ndash;Solomon Codes and Berlekamp&ndash;Welch Decoder:**  You will implement basic encoding and decoding methods for Reed&ndash;Solomon Codes as polynomial codes.

2. **Extended Euclidean Algorithm:**  You will implement the generalized Euclidean algorithm and, as an application, you will find the multiplicative inverse of nonzero elements in large finite fields.

3. **Generalized Reed&ndash;Solomon Codes:**  You will learn about Generalized Reed&ndash;Solomon Codes and implement an efficient decoder for them.

4. **Bose&ndash;Chaudhuri&ndash;Hocquenghem codes:** You will learn how to decode BCH codes by considering them as subfield subcodes of Generalized Reed&ndash;Solomon codes.

Each exercise is prefaced by some introductory remarks to help you complete that exercise. As always, feel free to post your questions and comments on [piazza](https://q.utoronto.ca/courses/222007/external_tools/5714?display=borderless).

Let's get started!

# 0.  Polynomials

Let $R$ be any ring, and let $x$ be an **indeterminate** (a **variable**).  A **polynomial** is a formal expression of the form $$a(x) = a_0 x^0 + a_1 x^1 + \cdots + a_n x^n = \sum_{i=0}^n a_i x^i,$$ where $n$ is any natural number.  Here $a_0$, $a_1$, $\ldots$, $a_n$ are elements of $R$, called the **coefficients** of $a(x)$.  Each $a_i x^i$ is called a **term** of $a(x)$.  The set of all polynomials in $x$ with coefficients from $R$ is denoted as $R[x]$.  A term of the form $1 \cdot x^i$ is usually written as $x^i$ and terms with zero coefficient are often omitted.  Likewise the terms $a_0 x^0$ and $a_1 x^1$ are written as $a_0$ and $a_1 x$, respectively. Thus $1 + 2x + x^3$ corresponds to the formal expression $1x^0 + 2x^1 + 0 x^2 + 1 x^3$.

Polynomials can be added by adding the coefficients of like terms (where missing terms are treated as having zero coefficient).  The **zero polynomial**, i.e., the polynomial all of whose coefficients are zero, serves as the additive identity.  Polynomials can be multiplied by assuming that the distributive law holds, simplifying terms of the form $(a_i x^i)\cdot (b_j x^j)$ to $a_i b_j x^{i+j}$.  The polynomial $1 = 1x^0$ serves as the multiplicative identity.  Under these operations, $R[x]$ itself forms a ring.

Since $R[x]$ is itself a ring, it is possible to form a ring $(R[x])[y]$ of polynomials in indeterminate $y$ having elements of $R[x]$ as coefficients.  One might also consider the ring $(R[y])[x]$ of polynomials in
interminate $x$ have elements of $R[y]$ as coefficients.  There is an obvious correspondence between these rings, either one of which is usually denoted as $R[x,y]$.  The elements of $R[x,y]$ are referred to as **bivariate** polynomials (to distinguish them from the **univariate** polynomials of $R[x]$).  One can extend this idea in the obvious way to form the **trivariate** ring $R[x,y,z]$, or, for any positive integer $m$, to form the **multivariate** ring $R[x_1,x_2,\ldots,x_m]$.


The **degree** of $a(x) = a_0 + a_1 x + \cdots + a_n x^n \in R[x]$, denoted as $\deg(a)$, is the largest index $i$ for which $a_i\neq 0$.  This definition does not work for the zero polynomial (since no such index exists), thus by convention the degree of the zero polynomial is defined to be $-\infty$.  Under this convention, provided that $R$ is an integral domain, we have
$$\deg(pq) = \deg(p) + \deg(q)$$ for all polynomials $p(x), q(x)  \in R[x]$.  (Caution:  addition of degrees doesn't generally hold in $R[x]$ when $R$ is a ring having zero divisors!)   A degree $d$ polynomial $a(x) = a_0 + a_1 x + \cdots + a_d x^d \in R[x]$ is said to be **monic** if $a_d = 1$.

In $R[x,y]$ we can view a bivariate polynomial as an element of $(R[y])[x]$ and assign it a degree (called
its $x$-degree).  Alternatively we can view it as an element of $(R[x])[y]$ and assign it a $y$-degree.  For example, consider $a(x,y) = a_0 + 2x + y + 3xy + xy^2$.  This polynomial can be written as $(a_0 + y) + (2 + 3y + y^2 )x \in (R[y])[x]$, from which we see that it has $x$-degree equal to one.  Alternatively, $a(x)$ can be written as $(a_0 + 2x) + (1 + 3x)y + (x)y^2 \in (R[x])[y]$, from which we see that it has $y$-degree equal to two.

In this numerical exercise, we will focus on $\mathbb{F}_q[x]$, i.e.,
on univariate polynomials with coefficients from the finite field $\mathbb{F}_q$. In the following, when we refer to a polynomial, we mean a univariate polynomial, unless clearly stated otherwise.  (Bivariate polynomials will appear in the context of the Berlekamp&ndash;Welch decoding procedure.)

Associated with a polynomial $p(x)\in\mathbb{F}_q[x]$ is a function defined by evaluation: $$\hat{p}:\mathbb{F}_q\to \mathbb{F}_q$$ $$\alpha\mapsto p(\alpha).$$

For simplicity, we use $p$ instead of $\hat{p}$ when talking about the function obtained by evaluation of the polynomial $p(x)$. This is an abuse of notation as these are essentially different objects. For example, consider the polynomials $p_1(x), p_2(x)\in\mathbb{F}_2[x]$ defined by $p_1(x) = x$ and $p_2(x) = x^2$. These are two distinct polynomials while the functions $\hat{p}_1$ and $\hat{p}_2$ are the same! If the evaluation of a polynomial $p(x)$ at a point $\alpha$ is zero, then $\alpha$ is called a **root** or **zero** of $p(x)$. 

In software, a polynomial of degree $d$ can be represented with a vector of length $d+1$. In particular, there is a `Polynomials` package in julia that uses this form of representation and allows us to perform basic polynomial operations. Run the following cell to load the `Galois2` module and install (if needed) and load the `Polynomials` package.

In [ ]:
include("Galois2.jl"); using .Galois2
#using Pkg; Pkg.add("Polynomials")     # uncomment if you need to install Polynomials for the first time.
using Polynomials

A polynomial can be constructed by calling the function `Polynomial`  from its vector of coefficients, lowest order first.

In [ ]:
a = GF2[1, 0, 1, 1]
p = Polynomial(a)

You can also provide the variable for the polynomial. The default variable is `x`:

In [ ]:
q = Polynomial(a, :x)
p == q

In [ ]:
r = Polynomial(a, :y)

In [ ]:
p == r

The evaluation of a polynomial can be performed by simply treating it as a function:

In [ ]:
p(GF2(0))

In [ ]:
p(GF2(1))

Let's create a primitive element in $\mathbb{F}_{2^8}$:

In [ ]:
α = gfprimitive(8) # (to type this, use \alpha+TAB)

In [ ]:
typeof(α)

In [ ]:
gforder(α) # a primitive element in Fq should have multiplicative order q-1

In [ ]:
isprimitive(α)

In [ ]:
isprimitive(α^3) # on the other hand, any power of alpha not relatively prime to q-1 is not primitive

In [ ]:
gforder(α^3)

In the following cells, some basic functionality provided by `Polynomials` is illustrated.

In [ ]:
# Create a polynomial from its roots
x = Polynomial(GF256[0,1])  # this gives the monomial x
roots = [α,α^2,α^3,α^4,α^5]
p = prod((x - r) for r in roots)

In [ ]:
degree(p) # compute the degree of p

**Caution:** The package `Polynomials` has a different convention for defining the degree of the zero polynomial and sets $\deg(0) = -1$. This way, the identity
$$
\deg(pq) = \deg(p) + \deg(q)
$$
only holds when both $p(x)$ and $q(x)$ are nonzero. In case you need to use `degree` function, beware of this convention.


In [ ]:
degree(Polynomial(zero(GF256)))

In [ ]:
cp = coeffs(p)  # extract the coefficients of p into a vector

In [ ]:
x = cp[1]
cp[1] = zero(GF256) # Caution: modifying the coefficient vector..
p  # modifies the polynomial!

In [ ]:
cp[1] = x # restore the polynomial
p

In [ ]:
p(α)  # polynomial evaluation

In [ ]:
p(one(GF256))  # polynomial evaluation

In [ ]:
typeof(p(one(GF256)))  # be careful to note that the value is *not* an integer!

In [ ]:
gflog(p(one(GF256))) # compute the discrete log of the previous result

In [ ]:
α^(gflog(p(one(GF256))))  # exp(log()) should be an identity function on nonzero elements

In [ ]:
q = x - α^6
p * q    # polynomial multiplication

In [ ]:
g = Polynomial(GF2[1,1,0,1])  # a generator polynomial for the (7,4) cyclic Hamming code
u = Polynomial(GF2[1,0,0,1])  # a message polynomial
x = Polynomial(GF2[0,1])      # the monomial x
q = div(x^3 * u, g)           # find the quotient of x^3 * u and g

In [ ]:
r = rem(x^3 * u, g)         # find the remainder after dividing x^3* u

In [ ]:
q*g + r  # should be x^3 * u

In [ ]:
v = x^3 * u - r  # form a Hamming codeword with message bits in higher order positions

In [ ]:
coeffs(v)  # extract the codeword symbols

In [ ]:
rem(v,g) == Polynomial(zero(GF2))  # let's check if this is a valid codeword.  If so, it must be a multiple of g(x)

** CAUTION ** The implementors of the `Polynomials` package sometimes treat constant polynomials (polynomials of degree zero) as scalars, but they make an incorrect assumption about the underlying field.  For example, try running the following cell.

In [ ]:
rem(v,Polynomial(one(GF2)))  # divide v(x) by one

To avoid this issue, we suggest adding the following new `safediv` and `saferem` methods for polynomials over finite fields of characteristic two.  These assume that the indeterminate is `x`.

In [ ]:
function safediv(a::Polynomial{<:Gf2,:x},b::Polynomial{<:Gf2,:x})
    T = typeof(coeffs(a)[1])
    x = Polynomial([zero(T),one(T)])
    div(x*a,x*b)
end

In [ ]:
function saferem(a::Polynomial{<:Gf2,:x},b::Polynomial{<:Gf2,:x})
    T = typeof(coeffs(a)[1])
    x = Polynomial([zero(T),one(T)])
    div(rem(x*a,x*b),x)
end

In [ ]:
safediv(v,Polynomial([one(GF2)]))

In [ ]:
saferem(v,Polynomial([one(GF2)]))

In the cell below, we form all polynomials in $\mathbb{F}_2^{<2}[x]$, i.e., all polynomials of degree at most 1 in $\mathbb{F}_2[x]$.  This is a vector space of dimension 2 over $\mathbb{F}_2$.

In [ ]:
p0 = Polynomial(GF2[0])
p1 = Polynomial(GF2[1])
p2 = Polynomial(GF2[0, 1])
p3 = Polynomial(GF2[1, 1])
P_1 = [p0, p1, p2, p3]
println.(P_1);

Next, we plot the functions associated with each the polynomials defined in the above cell:

In [ ]:
using Plots
x = [0, 1]
y = zeros(2, 4)
for k = 1:4
    y[:, k] = map(x -> x == GF2(0) ? 0 : 1, P_1[k].(GF2.(x)))
end
allplots = [plot(x, y[:, k], seriestype = :scatter, title = "$(P_1[k])", legend = false, 
        xlim = (-0.2, 1.2), ylim = (-0.2, 1.2), xticks = 0:1, yticks = 0:1) for k = 1:4]
plot(allplots..., layout = (2, 2))


Notice that we obtained all possible functions from $\mathbb{F}_2$ to $\mathbb{F}_2$ by considering the set of functions associated with $\mathbb{F}_2^{<2}[x]$. 

**Remark:** In fact, there is a one-to-one correspondence between the set of functions from $\mathbb{F}_q$ to $\mathbb{F}_q$ and the set of polynomial functions $\mathbb{F}_q^{<q}[x]$.

Let us now create the linear polynomial $p(x) = 1+x$ over $\mathbb{F}_4$ and plot its associated function:

In [ ]:
a = GF4[1, 1]
p = Polynomial(a)
x = [0, 1, 2, 3]
y = map(x -> x == GF4(0) ? 0 : (x == GF4(1) ? 1 : x == GF4(2) ? 2 : 3), p.(GF4.(x)))

plot(x, y, seriestype = :scatter, title = "$(p)", legend = false, 
        xlim = (-0.2, 3.2), ylim = (-0.2, 3.2), xticks = 0:3, yticks = 0:3)

Let us now consider $q(x) = p^2(x) = (1+x)^2 = 1 + x^2$ over $\mathbb{F}_4$ and plot its associated function:

In [ ]:
q = p^2
y = map(x -> x == GF4(0) ? 0 : (x == GF4(1) ? 1 : x == GF4(2) ? 2 : 3), q.(GF4.(x)))

plot(x, y, seriestype = :scatter, title = "$(q)", legend = false, 
        xlim = (-0.2, 3.2), ylim = (-0.2, 3.2), xticks = 0:3, yticks = 0:3)

## **<font color=green>Exercise 0: This exercise will not be marked and merely serves the purpose of familiarizing you with `Polynomials`. </font>**

---

1. Evaluate the polynomial $p(x) = x^{256} - x$ at all elements of $\mathbb{F}_{256}$. For how many elements do you get 0?  *Programming note:*  if `v` is a vector of elements from `GF256`, then `count(x->x==zero(GF256),v)` counts the number of zero coordinates.

2. Implement a function `conjugates(a::Gf2)` that takes a field element `a` from $\mathbb{F}_{2^m}$ and returns a vector containing the conjugates of `a` with respect to $\mathbb{F}_{2}$. Test your algorithm by computing the conjugates of a primitive element in $\mathbb{F}_{2^m}$ for each $m \in \{ 1, 2, \ldots, 8 \}$.

    **Hint:** Recall that the set of conjugates of an element $a\in\mathbb{F}_{2^m}$ with respect to $\mathbb{F}_2$ is $$\left\{a, a^2, a^{2^2}, a^{2^3}, \dots\right\}.$$ 

3. Using your implementation in part 2, implement a function `minpoly(a::Gf2)` that takes, as input, a field element `a` from $\mathbb{F}_{2^m}$ and returns the minimal polynomial of `a` with respect to $\mathbb{F}_{2}$.  For later use, ensure that your function returns a value of type `Polynomial{Gf2_1,:x}`.  Test your algorithm by computing the minimal polynomial of a primitive element in $\mathbb{F}_{2^m}$ for each $m \in \{ 1, 2, \ldots, 8 \}$.

    **Hint:** The minimal polynomial for an element $a\in\mathbb{F}_{2^m}$ with respect to $\mathbb{F}_2$ is the monic polynomial in $\mathbb{F}_2[x]$ of smallest degree having $a$ as a root, and can be found using $$M_a(x) = \prod_{\gamma\in C(a)}(x-\gamma)$$ where $C(a)$ is the set of conjugates of $a$ with respect to $\mathbb{F}_2$.
    
    *Programming note:*  `zero(typeof(a))` and `one(typeof(a))` return 0 and 1 elements in the same field as `a`.

In [ ]:
# answer for 0.1 here

In [ ]:
function conjugates(a::Gf2)
    # fill in the rest
end

In [ ]:
for m in 1:8
    println("$(m): $(conjugates(gfprimitive(m)))")
end

In [ ]:
function minpoly(a::Gf2)
    T = typeof(a)
    # fill in the rest
end

In [ ]:
typeof(minpoly(gfprimitive(3)))  # you should get Polynomial{Gf2_1,:x}

In [ ]:
for m in 1:8
    println("$(m): $(minpoly(gfprimitive(m)))")
end

***

# 1. Reed&ndash;Solomon Codes and Berlekamp&ndash;Welch Decoder
Let $\mathbb{F}_q$ be a field of size $q$. For $k \geq 1$, let $\mathbb{F}_q^{<k}[x]$ denote the set of polynomials of
degree at most $k − 1$ over $\mathbb{F}_q$. We note that $\mathbb{F}_q^{<k}[x]$ is a vector space of dimension $k$
over $\mathbb{F}_q$, with basis $\{1, x, x^2, \dots , x^{k−1}\}$. This vector space contains $q^k$ different polynomials, each of the form
$$
u(x) = \sum_{i=0}^{k-1}u_ix^i,\quad u_i\in\mathbb{F}_q.
$$

Let $\mathcal{E} = (\alpha_1, \alpha_2, \dots, \alpha_n)$ be some ordered subset of distinct elements of $\mathbb{F}_q$ called the **code locators**. The evaluation map
$$
\mathtt{ev}_{\mathcal{E}}:\mathbb{F}_q^{<k}\to\mathbb{F}_q^n
$$
with respect to $\mathcal{E}$ sends a polynomial $u(x)$ to the $n$-tuple
$$
\left(u(\alpha_1), u(\alpha_2),\dots, u(\alpha_n)\right).
$$

This evaluation map is a linear map. In fact, if $n\geq k$ the map $\mathtt{ev}_{\mathcal{E}}$ is injective. Therefore, the image of this linear map is a subspace of $\mathbb{F}_q^n$ of dimension $k$. Denote this image by $C$, i.e., 
$$
C = \mathtt{ev}_{\mathcal{E}}\left(\mathbb{F}_q^{<k}\right).
$$
Hence, $C$ is an $(n,k)$ code called a Reed&ndash;Solomon (RS) code of dimension $k$ with code locators $\mathcal{E}$. By considering the fact that each nonzero polynomial $u(x)\in\mathbb{F}_q^{<k}$ has at most $k-1$ roots, it follows that the minimum weight of the nonzero codewords in this RS code is at least $n-(k-1)$. That is, RS codes are MDS!

To encode a message vector $u\in\mathbb{F}_q^k$ according to this RS code, we first associate the message vector with the message polynomial
$$
u(x) = \sum_{i=0}^{k-1}u_ix^i
$$

and then find the corresponding codeword by using the evaluation map $\mathtt{ev}_{\mathcal{E}}$.

The Berlekamp-Welch (BW) algorithm provides a way to decode RS codes. Let $u$ be a message vector that is mapped to the codeword $c$ via the evaluation map $\mathtt{ev}_\mathcal{E}$ where $\mathcal{E} = (\alpha_1,\ldots,\alpha_n)$ for distinct code locators $\alpha_,\ldots,\alpha_n \in \mathbb{F}_q$. Let the received word be $y = c + e$ where $e$ is the error vector which we assume has a Hamming weight of at most $t = \frac{n-k}{2}$.

The BW algorithm first finds a bivariate polynomial $Q(x,y)$ of the form
$$
Q(x,y) = Q_0(x) + yQ_1(x), \quad \deg(Q_0)\leq n-t-1,\quad \deg(Q_1)\leq n-t-k
$$
that **interpolates** the pairs
$$
P = \{(\alpha_1, y_1), (\alpha_2, y_2), \dots, (\alpha_n, y_n)\}
$$
in a way that 
$$
Q(\alpha_i,y_i) = 0,\quad \forall (\alpha_i,y_i)\in P.
$$

This latter equation gives a system of linear equations in the coefficients of $Q_0$ and $Q_1$ that is guaranteed to have solutions and can be solved using standard techniques from linear algebra.  More precisely, denote the (unknown) coefficients of $Q_0(x)$ as $a_0, a_1, \ldots, a_{n-t-1}$ and denote the (unknown) coefficients of $Q_1(x)$ as $b_0, b_1, \ldots, b_{n-t-k}$.  The number of unknowns is $$(n-t) + (n - t -k +1) = n + (n-2t -k) +1 = \begin{cases} n+1 & \text{if }n-k=2t,\\ n+2 & \text{if }n-k=2t+1. \end{cases}$$
The interpolation condition gives $n$ homogeneous linear equations in these unknowns, which can be written in matrix form as follows:
$$
\left[\begin{array}{cccccccccc}
1 & \alpha_1 & \alpha_1^2 & \cdots & \alpha_1^{n-t-1} & y_1 & y_1 \alpha_1 & y_1 \alpha_1^2 & \cdots & y_1 \alpha_1^{n-t-k} \\
1 & \alpha_2 & \alpha_2^2 & \cdots & \alpha_2^{n-t-1} & y_2 & y_2 \alpha_2 & y_2 \alpha_2^2 & \cdots & y_2 \alpha_2^{n-t-k} \\
1 & \alpha_3 & \alpha_3^2 & \cdots & \alpha_3^{n-t-1} & y_3 & y_3 \alpha_3 & y_3 \alpha_3^2 & \cdots & y_3 \alpha_3^{n-t-k} \\
1 & \alpha_4 & \alpha_4^2 & \cdots & \alpha_4^{n-t-1} & y_4 & y_4 \alpha_4 & y_4 \alpha_4^2 & \cdots & y_4 \alpha_4^{n-t-k} \\
\vdots & \vdots & \vdots &  & \vdots  & \vdots & \vdots & \vdots & \vdots &  & \vdots \\
1 & \alpha_n & \alpha_n^2 & \cdots & \alpha_n^{n-t-1} & y_n & y_n \alpha_n & y_n \alpha_n^2 & \cdots & y_n \alpha_n^{n-t-k}
\end{array}\right] \left[\begin{array}{c} a_0 \\ \vdots \\ a_{n-t-1} \\ b_0 \\ \vdots \\ b_{n-t-k} \end{array}\right] = \left[ \begin{array}{c} 0 \\ 0 \\ 0 \\ 0 \\ \vdots \\ 0 \end{array}\right]
$$
**Hint:** to find a nonzero solution, consider the given matrix to be the parity-check matrix for a linear code of length $2n -2t -k +1$ over $\mathbb{F}_q$.  You seek a nonzero codeword.  Consider converting the parity-check matrix to a generator matrix, perhaps re-using a previously written function (from Numerical Exercise 1).

The BW algorithm is then described as follows:
> **Input:** received word $y = (y_1, y_2, \dots, y_n)$; code locators $\mathcal{E} = (\alpha_1, \dots, \alpha_n)$ and the dimension of the code $k$
>
> **Output:** message polynomial $u(x)$
>
>  1. Form a bivariate polynomial $Q(x,y) = Q_0(x) + yQ_1(x)$ as described.
>
>  2. Form $f(x) = -\frac{Q_0(x)}{Q_1(x)}$.  If a nonzero remainder results, declare a decoding failure.
>
>  3. Check that the Hamming distance between $y$ and $\mathtt{ev}_{\mathcal{E}}(f(x))$ is at most $\frac{n-k}{2}$; otherwise declare a decoding failure.
>  4. Return $f(x)$

## **<font color=green>Exercise 1:</font>**

---

1. Implement a function `rs_encoder(u::AbstractVector{<:Gf2}, E::AbstractVector{<:Gf2})` that takes a information vector `u` of length $k$ as well as a vector of code locators `E` of length $n$ and produces the corresponding codeword `c` in the $(n, k)$ RS code defined with code locators `E`.

2. Implement the Berlekamp&ndash;Welch decoder, as described above, by providing a function `bw_decoder(y::AbstractVector{<:Gf2},E::AbstractVector{<:Gf2},k::Int)` that takes a received word `y`, code locators `E` and the code dimension `k`. Your implemented function must return the correct message polynomial if `y` is at a Hamming distance of at most $t = \frac{n-k}{2}$ from a valid codeword. Your decoder must declare failure (by returning an empty vector) in case the division in the second step 2 of BW algorithm is not possible or if the condition in step 3 of BW algorithm is not satisfied.

3. Test your implementation by running the following cell.


In [ ]:
function rs_encoder(u::AbstractVector{<:Gf2},E::AbstractVector{<:Gf2})
    # fill in the rest
end

In [ ]:
function dual(G::AbstractArray{<:Gf2})  # convert from G to H or vice-versa
   # you can copy/modify this from Numerical Exercise 1
end

In [ ]:
function bw_decoder(y::AbstractVector{<:Gf2},E::AbstractVector{<:Gf2},k::Int)
    # fill in the rest
end

In [ ]:
# Test your implementation.
message = GF256[73,116,32,119,111,114,107,101,100,33,32,32,67,111,110,103,114,97,116,117,108,
    97,116,105,111,110,115,32,111,110,32,105,109,112,108,101,109,101,110,116,105,110,103,
    32,121,111,117,114,32,102,105,114,115,116,32,82,101,101,100,45,83,111,108,111,109,111,
    110,32,100,101,99,111,100,101,114,33]
k = length(message)
t = 30 # we will make a t-error-correcting code
n = k + 2*t
if n > 255
    println("Oops, excessive block-length!")
end
E = [gfprimitive(8)^i for i=1:n]
p = 0.2  # probability of symbol error
e = [rand() < p ? GF256(rand(1:255)) : zero(GF256) for i = 1:n]
w = count(x->x!=zero(GF256),e)
print("Error pattern has weight $(w), which ")
if w <= t
    println("should decode correctly.")
else
    println("will likely cause a decoding failure.")
end
c = rs_encoder(message,E)
u_hat = bw_decoder(c+e,E,k)
if length(u_hat) > 0
    println(String([Char(u_hat[i].value) for i = 1:length(u_hat)]))
else
    println("Decoding failure!")
end


---


# 2. The Extended Euclidean Algorithm

The extended Euclidean algorithm is an extension to Euclid's Algorithm that computes, in addition to the greatest common divisor (gcd) of two elements $a$ and $b$ (which are not both zero) in a Euclidean domain, also the coefficients $s$ and $t$ of Bézout's identity so that
$$
\gcd(a, b) = s\cdot a + t\cdot b.
$$

Define the norm of a polynomial $p(x)\in\mathbb{F}_q[x]$ as its degree, $N(p) = \deg(p)$. Equipped with $N(\cdot)$, the ring of polynomials $\mathbb{F}_q[x]$ becomes a Euclidean domain. Naturally, the extended Euclidean algorithm for two polynomials $a(x)$ and $b(x)$ can be used to find the greatest common divisor of $a(x)$ and $b(x)$ as well as two polynomials $s(x)$ and $t(x)$ such that
$$
\gcd(a, b) = s(x)a(x) + t(x)b(x).
$$

A pseudocode description of the extended  Euclidean algorithm is given below (see Algorithm 47 in Chapter 7 of the Lecture Notes):

> **Require:** $a,b\in E, a\neq 0, N(a)\geq N(b)$ or $b=0$, where $E$ is a Euclidean domain.
>
> $M \leftarrow \begin{bmatrix}a & 1 & 0\\b & 0 & 1\end{bmatrix}$ \{Notation: $M = [m_{ij}]$\}
>
> **while $m_{21}\neq 0$ **do**
>
> Using the division algorithm in $E$, find $q$ such that $m_{11} = qm_{21} + r$ with $r = 0$ or $N(r) < N(m_{21})$;
>
> $M \leftarrow \begin{bmatrix}0 & 1\\1 & -q\end{bmatrix}M$
>
> **end while**
>
> **return** $(gcd(a,b), s, t) = (m_{11}, m_{12}, m_{13})$

## **<font color=green>Exercise 2:</font>**

---

1. Implement a function `(g, s, t) = eea(a, b)` that takes two polynomials `a` and `b` both over the same finite field $\mathbb{F}$ and returns the greatest common divisor of `a`  and  `b` (called `g`) as well as two polynomials `s` and `t` such that `g = a * s + b * t`. In case both inputs are zero, your implementation must return `(0, 1, 0)`.  **Reminder:** use the `safediv` and `saferem` functions (as needed) that were defined earlier.

2. Let $g(x) = 1 + x + x^{127} \in \mathbb{F}_2[x]$.  Since $g(x)$ is irreducible, the quotient ring $\mathbb{F}_2[x]/\langle g(x) \rangle$ is the finite field $\mathbb{F}_{2^{127}}$.  Let $a(x) = 1 + x + x^2 + x^3 + x^4$. Find the multiplicative inverse of the elements $[a(x)]$  and $[1 + x^{122}a(x)]$ in this field, and verify that the elements that you have found are indeed the multiplicative inverses.   **Hint:** If $b(x)$ is any polynomial of degree < 127, then $\gcd(b, g) = 1$.  Use your implementation of `eea` to find Bézout's coefficients.

In [ ]:
function eea(a,b)
    # fill in the rest
end        

In [ ]:
eea(Polynomial(zero(GF2)),Polynomial(zero(GF2)))

In [ ]:
# create cells as needed to complete exercise 2.2


---


# 3. Syndrome Decoding of Generalized Reed&ndash;Solomon Codes
Let $\mathbb{F}_q$ be a finite field and let $\mathcal{E}=(\alpha_1, \alpha_2, \dots, \alpha_n)$ be an ordered set of distinct elements from $\mathbb{F}_q$ called code locators. Let $\mathcal{M} = (\mu_1, \mu_2, \dots, \mu_n)$ be an ordered set of nonzero (not necessarily distinct) elements from $\mathbb{F}_q$ called column multipliers. 

A generalized Reed&ndash;Solomon (GRS) code can be specified by giving either a generator matrix or a parity-check matrix in the form
$$
\begin{bmatrix}
1 & 1 & 1 & \dots & 1\\
\alpha_1 & \alpha_2 & \alpha_3 & \dots & \alpha_n\\
\alpha_1^2 & \alpha_2^2 & \alpha_3^2 & \dots & \alpha_n^2\\
\vdots & \vdots & \vdots & \ddots & \vdots\\
\alpha_1^{r-1} & \alpha_2^{r-1} & \alpha_3^{r-1} & \dots & \alpha_n^{r-1}\\
\end{bmatrix} \mathit{\mathrm{diag}}\left(\mu_1, \mu_2, \mu_3, \dots, \mu_n\right)
$$

where $\mathit{\mathrm{diag(\cdot)}}$ denotes a diagonal matrix with the given entries on the diagonal.

Taken as a generator matrix, the above equation defines an $(n,r)$ GRS code, and the column multipliers are then referred to as generator-matrix
column multipliers. Taken as a parity-check matrix, it defines
an $(n, n-r)$ GRS code, and the column multipliers are then referred to as parity-check-matrix column multipliers.



### 3.1 Encoding
Encoding of a GRS code with the generator matrix
$$
G = \begin{bmatrix}
1 & 1 & 1 & \dots & 1\\
\alpha_1 & \alpha_2 & \alpha_3 & \dots & \alpha_n\\
\alpha_1^2 & \alpha_2^2 & \alpha_3^2 & \dots & \alpha_n^2\\
\vdots & \vdots & \vdots & \ddots & \vdots\\
\alpha_1^{k-1} & \alpha_2^{k-1} & \alpha_3^{k-1} & \dots & \alpha_n^{k-1}\\
\end{bmatrix} \mathit{\mathrm{diag}}\left(\mu_1, \mu_2, \mu_3, \dots, \mu_n\right)
$$
can be accomplished by means of polynomial evaluation. For an input word
$$ u = (u_0, u_1, \dots, u_{k-1})$$

define a corresponding polynomial $u(x)$ of degree at most $k-1$ by 
$$
u(x) = u_0 + u_1x + \dots + u_{k-1}x^{k-1}.
$$
You can easily verify that
$$
uG = \left(\mu_1u(\alpha_1), \mu_2u(\alpha_2), \dots, \mu_nu(\alpha_n)\right).
$$


## **<font color=green>Exercise 3 (Part I: Encoders)</font>**


1. Implement a function `grs_encoder(u::AbstractVector{<:Gf2},E::AbstractVector{<:Gf2},M::AbstractVector{<:Gf2}))` that takes an information vector `u`, a vector of distinct code locators `E`, a vector of generator-matrix column multipliers `M` and returns the corresponding codeword.  Run the following cells to verify your encoder.
2. Implement a function `M2M(E::AbstractVector{<:Gf2},M::AbstractVector{<:Gf2})` that takes a vector of distinct code locators `E` and a vector of generator-matrix column multipliers `M` and returns the corresponding parity-check-matrix column multipliers.
    **Hint:** See the proof of Theorem 15 in Chapter 6 of the lecture notes. In that proof you will observe that you seek a nonzero codeword in a linear code of length $n$ given by a parity-check matrix. Consider converting the parity-check matrix to a generator matrix, perhaps re-using a previously written function (from Numerical Exercise 1).

In [ ]:
function grs_encoder(u::AbstractVector{<:Gf2},E::AbstractVector{<:Gf2},M::AbstractVector{<:Gf2})
    # fill in the rest
end

In [ ]:
grs_encoder(GF8[0,1,0],GF8[1,2,3,4,5,6,7],GF8[7,3,3,1,6,6,4])  # expect [7,6,5,4,3,2,1]

In [ ]:
function M2M(E::AbstractVector{<:Gf2},M::AbstractVector{<:Gf2})
    # fill in the rest
end

In [ ]:
M2M(GF8[1,2,3,4,5,6,7],GF8[7,3,3,1,6,6,4])  # expect (5,4,6,5,5,6,1)

In [ ]:
v = grs_encoder(GF8[2,4,6],GF8[1,2,3,4,5,6,7],GF8[7,3,3,1,6,6,4])  # a "random" codeword of C
w = grs_encoder(GF8[1,3,5,7],GF8[1,2,3,4,5,6,7],M2M(GF8[1,2,3,4,5,6,7],GF8[7,3,3,1,6,6,4])) # and of the dual
sum(+,[v[i]*w[i] for i in 1:length(v)])  # compute their inner product; the result should be...

### 3.2 Syndrome Decoding
Let $\mathcal{C}$ be an $(n,k, d = n-k+1)$ GRS code over a finite field $\mathbb{F}_q$ with parity check matrix
$$
H = \begin{bmatrix}
1 & 1 & 1 & \dots & 1\\
\alpha_1 & \alpha_2 & \alpha_3 & \dots & \alpha_n\\
\alpha_1^2 & \alpha_2^2 & \alpha_3^2 & \dots & \alpha_n^2\\
\vdots & \vdots & \vdots & \ddots & \vdots\\
\alpha_1^{d-2} & \alpha_2^{d-2} & \alpha_3^{d-2} & \dots & \alpha_n^{d-2}\\
\end{bmatrix} \mathit{\mathrm{diag}}\left(\mu_1, \mu_2, \mu_3, \dots, \mu_n\right)
$$
where the code locators $(\alpha_1, \alpha_2, \dots, \alpha_n)$ are distinct and **nonzero**.  For the syndrome-based decoder that we will now develop, which involves the reciprocals of the code locators, the zero code-locator is not permitted!  Notice that the $(\mu_1, \ldots, \mu_n)$ now represent **parity-check column multipliers** rather than generator matrix column multipliers.  Luckily we have a convenient function that takes us back and forth between them!

Suppose a codeword $c$ is being transmitted and the received word $y$ is given by
$y = c + e $
where $e = (e_0, e_1, \dots, e_{n-1})$ is the error vector. We assume that the Hamming weight $e$ is at most $(d-1)/2$.

The **syndrome** corresponding to the received word $y = (y_0,y_1, \dots, y_{n-1})$  is 
$$
s = yH^T = (s_0, s_1, \ldots, s_{d-2})
$$
where
$$
s_i = \sum_{j=1}^n y_j \mu_j \alpha_j^i.
$$
Associate with the syndrome the **syndrome polynomial** $$s(x) = s_0 + s_1 x + \cdots + s_{d-2} x^{d-2}.$$

## **<font color=green>Exercise 3 (Part II: Syndromes)</font>**


1. Implement a function `grs_syndrome(y::AbstractVector{<:Gf2},E::AbstractVector{<:Gf2},M::AbstractVector{<:Gf2},k::Int)` that takes a received vector `y`, a vector of distinct code locators `E`, a vector of parity-check column multipliers `M` and the code dimension `k`, and returns the corresponding syndrome polynomial.  Run the following cells to verify your syndrome-former.

In [ ]:
function grs_syndrome(y::AbstractVector{<:Gf2},E::AbstractVector{<:Gf2},M::AbstractVector{<:Gf2},k::Int)
   # fill in the rest
end            

In [ ]:
grs_syndrome(GF8[7,6,5,4,3,2,1],GF8[1,2,3,4,5,6,7],GF8[5,4,6,5,5,6,1],3)  # this is a codeword!

In [ ]:
grs_syndrome(GF8[7,6,5,4,3,2,0],GF8[1,2,3,4,5,6,7],GF8[5,4,6,5,5,6,1],3)  # error in last position

In [ ]:
grs_syndrome(GF8[7,6,5,4,3,0,0],GF8[1,2,3,4,5,6,7],GF8[5,4,6,5,5,6,1],3)  # error in last two positions

## 3.3 Solving the Key Equations for the Error-Locator and Error-Evaluator Polynomials

Now, since $y=c+e$, the syndrome $s$ is
$$
(c+e)H^T = eH^T = (s_0, s_1, \dots, s_{d-2})
$$
with
$$
s_i = \sum_{j = 1}^n e_j\mu_j\alpha_j^i = \sum_{j \in J} e_j\mu_j\alpha_j^i.
$$
where $J$ is the set of positions in which $e$ is nonzero, i.e., the locations of errors. Note, 
$$
\lvert J\rvert \leq \frac{d-1}{2}
$$
From this we see that
$$
s(x) = \sum_{j\in J}e_j\mu_j\sum_{i=0}^{d-2}(\alpha_jx)^i.
$$

If we consider polynomials modulo $x^{d-1}$, you easily verify that
$$
(1-\alpha_jx)\sum_{i=0}^{d-2}(\alpha_jx)^i = 1 - (\alpha_jx)^{d-1}  \equiv 1 \quad\mathrm{(mod~}x^{d-1}\mathrm{)}.
$$

Hence, we have
$$
s(x) \equiv \sum_{j\in J} \frac{e_j\mu_j}{1-\alpha_jx}\quad\mathrm{(mod~}x^{d-1}\mathrm{)}.
$$
where the polynomial division corresponds to multiplication by the corresponding multiplicative inverse in the polynomial ring $\mathbb{F}_q[x]/\langle x^{d-1} \rangle$.  (In this ring, an element $[a(x)]$ is invertible if and only if $a(0) \neq 0$.)


Recall the definition of the **error locator polynomial**
$$
\Lambda(x) = \prod_{j\in J}(1 - \alpha_jx)
$$
and the **error evaluator polynomial**
$$
\Gamma(x) = \sum_{j\in J} e_j\mu_j\prod_{m\in J\setminus\{j\}} (1-\alpha_mx).
$$

Note that the evaluation of the error locator polynomial at the reciprocal of a code locator corresponding to an error location is zero, hence the name. In other words, the roots of $\Lambda(x)$ are precisely the multiplicative inverses of code locators corresponding to the error locations.  The roots of the error locator can be found by testing the reciprocal of the code locators one-by-one (a linear search called a **Chien search**).

This also leads us to the first key equation:
$$
\gcd(\Lambda(x), \Gamma(x)) = 1
$$

Additionally, by inspection you can see that
$$\deg(\Gamma)<\deg(\Lambda) = \lvert J\rvert \leq \frac{d-1}{2}$$

which will be the second key equation for us.

Define the formal derivative of a polynomial $p(x) = \sum_{k=0}^Lp_k x^k\in \mathbb{F}_q[x]$ by
$$
p'(x) = \sum_{k=1}^L\bar{k}p_k x^{k-1}
$$
with 
$$
\bar{k} = \underbrace{1 + 1 + \dots + 1}_{m~\mathrm{terms}},\quad 1 \mathrm{~is~the~identity~element~of~}\mathbb{F}_q.
$$
Again, by inspection, you can see that
$$
\Lambda(x)s(x)\equiv\Gamma(x)\quad\mathrm{(mod~}x^{d-1}\mathrm{)}
$$
which is the third key equation.

The value of the error $e_j$, for $j\in J$, can be found using **Forney's formula** by
$$
e_j = -\frac{\alpha_j}{\mu_j}\frac{\Gamma(a_j^{-1})}{\Lambda'(a_j^{-1})}.
$$

The main step in decoding GRS codes, therefore, is to find the error locator polynomial and the error evaluator polynomial. This is accomplished by solving the key equations which are given here again for convenience:
$$
\gcd(\Lambda(x), \Gamma(x)) = 1
$$
$$
\deg(\Gamma)<\deg(\Lambda)\leq \frac{d-1}{2}
$$
$$
\Lambda(x)s(x)\equiv\Gamma(x)\quad\mathrm{(mod~}x^{d-1}\mathrm{)}
$$

The key equations can be solved using the extended Euclidean algorithm, with a different stopping criterion, which we call the *modified EEA* (see Chapter 8 of the lecture notes, or Chapter 6 of R. M. Roth, *Introduction to Coding Theory*, Cambridge University Press, 2006).  Suppose we run the EEA with $x^{d-1}$ and $s(x)$ as input.  At each stage of the algorithm, we have a remainder $r(x)$ which is expressed as a linear combination $a(x) x^{d-1} + b(x) s(x)$.  Reducing modulo $x^{d-1}$ we see that $r(x) = b(x)s(x) \bmod x^{d-1}$ and thus $r(x)$ is a candidate for $\Gamma(x)$ and $b(x)$ is a candidate for $\Lambda(x)$.  It can be proved that indeed $r(x) = \Gamma(x)$ and $b(x) = \Lambda(x)$ on the iteration of the EEA when the condition $\deg(r(x)) < t$ is first satisfied.  Thus the stopping criterion for the modified EEA is chosen so that as soon as $\deg(m_{21}) < t$, the algorithm exits the while loop (see the pseudocode given before Exercise 2).

## **<font color=green>Exercise 3 (Part III: Error-Locator and Error-Evaluator Polynomials)</font>**


1. Copy your implementation of `eea` from Exercise 2 and modify it so that a new function `mod_eea` is defined that takes in the syndrome polynomial $s(x)$ and the integer $d$ and returns the error-locator and error-evaluator polynomials $\Lambda(x)$ and $\Gamma(x)$ (as a tuple, in that order).  Verify your implementation by running the following cells.

In [ ]:
function mod_eea(s,d)
    # fill in the rest, copying from eea
end        

In [ ]:
E = GF8[1,2,3,4,5,6,7]
M = GF8[5,4,6,5,5,6,1]
n = length(E)
k = 3
d = n-k+1
s = grs_syndrome(GF8[7,6,5,4,3,2,1],E,M,k) # no errors
mod_eea(s,d)  # what degree do you expect for the error-locator?

In [ ]:
s = grs_syndrome(GF8[7,6,5,4,3,2,0],E,M,k) # one error
mod_eea(s,d)  # what degree do you expect for the error-locator?

In [ ]:
s = grs_syndrome(GF8[7,6,5,4,3,0,0],E,M,k) # two errors
mod_eea(s,d)  # what degree do you expect for the error-locator?

In [ ]:
s = grs_syndrome(GF8[7,6,5,4,0,0,0],E,M,k) # three errors
mod_eea(s,d)  # what degree do you expect for the error-locator?

## 3.4 Putting the Pieces Together

All of this leads to the following syndrome-based decoding algorithm for GRS codes with nonzero code locators.  Let $\mathcal{E} = (\alpha_1, \alpha_2, \ldots, \alpha_n)$ be the code locators and let $\mathcal{M} = (\mu_1, \mu_2, \ldots, \mu_n)$ be the parity-matrix column multipliers.

> **Input:** received word $y = (y_0, y_1, \dots, y_{n-1})$
>
> **Output:** error word $e = (e_0, e_1, \dots, e_{n-1})$
>
>  1. Compute the syndrome: compute the polynomial $s(x) = s_0 + s_1x+\dots+s_{d-2}x^{d-2}$ by using
>  $$s_i  = \sum_{j =0}^n y_j\mu_j\alpha_j^i.$$
>        
>  2. Find the error-locator polynomial $\Lambda(x)$ and the error-evaluator polynomial $\Gamma(x)$: use the modified EEA with inputs $x^{d-1}$ and $s(x)$ to obtain $g(x)$ as well as two polynomials $a(x)$ and $b(x)$ such that
>  $$  g(x) = a(x)x^{d-1} + b(x)s(x), $$
> stopping on the first iteration where $\deg(g(x))<t$. Set
>  $$  \Gamma(x) =  g(x), \quad \Lambda(x) =  b(x).$$
>
>  3. Finding the error locations and values: for each $j \in \{ 1, \ldots, n \}$ set
> $$ e_j = \begin{cases} - \frac{\alpha_j}{\mu_j} \frac{\Gamma\left(\alpha_j^{-1}\right)}{\Lambda'\left(\alpha_j^{-1}\right)} & \text{if } \Lambda(\alpha_j^{-1}) = 0 \\ 0 & \text{otherwise}. \end{cases} $$

## **<font color=green>Exercise 3 (Part IV): Building the Decoder</font>**

1. Implement a function `D(p)` which takes in a polynomial `p` in $\mathbb{F}_{2^m}[x]$ and returns its formal derivative.  Note that in any field of characteristic two, $1+1=0$, $1+1+1=1$, $1+1+1+1=0$, etc.  Test your function by running the following cell.

2. Implement a function `grs_decoder(y::AbstractVector{<:Gf2},E::AbstractVector{<:Gf2},M::AbstractVector{<:Gf2},k::Int)` that takes a received word `y`, which is a noisy version of some codeword `c`, the vector of distinct and nonzero code locators `E`, the vector of parity-check-matrix column multipliers `M` and the dimension of the code `k` and returns the error vector `e` obtained by syndrome decoding of GRS codes with solving the key equations.  To declare a decoding failure, the decoder should return an empty vector.  (A decoding failure will arise if the number of nonzero positions found for `e` does not match the degree of the error-locator polynomial, or if the Forney formula would give a divide-by-zero error.)  Test your implementation by running the following cells.


In [ ]:
function D(p)
    # fill in the rest
end

In [ ]:
D(Polynomial(GF8[1,2,3,4,5,6,7]))  # expect D(1 + 2x + 3x^2 + 4x^3 + 5x^4 + 6x^5 + 7x^6) = 2 + 4x^2 + 6x^4

In [ ]:
function grs_decoder(y::AbstractVector{<:Gf2},E::AbstractVector{<:Gf2},M::AbstractVector{<:Gf2},k::Int)
    # fill in the rest
end

In [ ]:
E = GF8[1,2,3,4,5,6,7]
M = GF8[5,4,6,5,5,6,1]
n = length(E)
k = 3
d = n-k+1
grs_decoder(GF8[7,6,5,4,3,2,1],E,M,k) # no errors

In [ ]:
grs_decoder(GF8[7,6,5,4,3,2,0],E,M,k) # one error, last position

In [ ]:
grs_decoder(GF8[7,6,5,4,3,0,0],E,M,k) # two errors, last two positions

In [ ]:
grs_decoder(GF8[7,6,5,4,0,0,0],E,M,k) # three errors, last three positions

In [ ]:
α = gfprimitive(8)
E = [ α^i for i in 0:254]  # let's make a cyclic GRS code this time
M = [ α^i for i in 0:254]
n = length(E)
k = 239  # the (255,239) RS code
t = div(n-k,2)
x = Polynomial(GF256[0,1])
g = prod( (x-α^i) for i in 1:n-k )  # the generator polynomial
println("Decoding the ($(n),$(k)) cyclic GRS code with generator polynomial $(g).")
TRIALS = 10000 # let's do this number of trials with errors in t random positions
FAILURES = 0
ERRORS = 0
for trial in 1:TRIALS
    u = GF256.(rand(0:255,k)) # generate a random k-symbol message
    v = coeffs(Polynomial(u)*g)  # encode by multiplying by the generator polynomial
    while length(v) < n  # it could happen that we get a low degree, so pad with zeros at the top
        push!(v,zero(GF256))
    end
    y = copy(v)  # uncorrupted received word
    for j in 1:t
        y[rand(1:n)] += GF256(rand(0:255))  # random error in a random position
    end
    e = grs_decoder(y,E,M,k)
    if length(e) == 0
        FAILURES += 1
    else
        y -= e
        if (y != v)
            ERRORS += 1
        end
    end
end
println("After $(TRIALS) trials, $(ERRORS) errors and $(FAILURES) failures were encountered.")


---


# 4. Binary BCH Codes

A cyclic GRS code $C_{RS}$ over a field $\mathbb{F}_{q^m}$ is a GRS code with code locators defined in terms of a primitive  $n$th root of unity $\alpha$ in $\mathbb{F}_{q^m}$, i.e., an element $\alpha$ of multiplicative order $n$.   The code has code locators $\mathcal{E} = (1, \alpha, \alpha^2, \dots, \alpha^{n-1})$ and parity-check-matrix column multipliers $\mathcal{M} = (1, \alpha^b, \alpha^{2b}, \dots, \alpha^{(n-1)b})$. Such a code is cyclic with generator polynomial
$$
g(x) = (x-\alpha^b)(x-\alpha^{b+1})\dots (x-\alpha^{b+d-2}),
$$
where $d$ is the minimum Hamming distance of the code. A vector $c\in \mathbb{F}_{q^m}^n$ is a codeword if and only if $g(x)\mid c(x)$ if and only if
$$
c(\alpha^b) = c(\alpha^{b+1}) = \dots = c(\alpha^{b+d-2}) = 0
$$
where $c(x)$ is the associated polynomial of the codeword $c$.

In general, if $C$ is a cyclic code over $\mathbb{F}_q$ (which is a subfield of $\mathbb{F}_{q^m}$) of length $n$ with generator polynomial $g(x)$ and there exist integers $b\geq 0$ and $\delta\geq 2$ such that
$$
g(\alpha^b) = g(\alpha^{b+1}) = \dots = g(\alpha^{b+\delta-2}) = 0,
$$
then $d_{\mathrm{min}}(C)\geq \delta$.

The minimum distance of the smallest RS code containing a given
cyclic code $C$ is called the **design distance** of $C$. The actual minimum distance of $C$ is at least as large as the design distance. Thus, if we wish to design a $t$-error correcting code, we choose $g(x)$ so that it has $2t$ consecutive zeros, i.e., design distance $2t + 1$. This is the main idea in development of Bose&ndash;Chaudhuri&ndash;Hocquenghem (BCH) codes. To design a BCH code over $\mathbb{F}_q$ of a minimum Hamming distance $\geq 2t+1$, we form the generator polynomial of the code by taking the product of as many minimal polynomials of powers of $\alpha$ in a way that $g(x)$ has $2t$ consecutive powers of $\alpha$ as its roots. Note, $\alpha$ is some primitive $n$th roots of unity (i.e., an element of multiplicative order $n$) in some extension field $\mathbb{F}_{q^m}$. An example is given below (see Chapter 8 of the Lecture Notes for more examples):

**Example:** Consider the class of binary cyclic codes of length 15. We can find a primitive 15th root of unity $\alpha$ as the primitive element in $\mathbb{F}_{16}$. The conjugacy classes in the integer powers of $\alpha$ with respect to $\mathbb{F}_2$ are
$$
\{\alpha^0\},\,\{\alpha^5, \alpha^{10}\},\,\{\alpha, \alpha^2, \alpha^4, \alpha^8\},\,\{\alpha^3, \alpha^6, \alpha^{12}, \alpha^{9}\},\,\{\alpha^7, \alpha^{14}, \alpha^{13}, \alpha^{11}\}.
$$

By taking $g(x) = M_{\alpha}(x)M_{\alpha^3}(x)$, we see that $g(\alpha^i) = 0$ for all $i\in\{1,2,3,4,8,9,12\}$ which contains 4 consecutive powers of $\alpha$. Hence, the binary BCH code obtained this way has a minimum distance of at least 5.


## **<font color=green>Exercise 4:</font>**

---

1. Implement a function `bch_generator(a::Gf2,t::Int,b::Int)` that takes an element `a` of multiplicative order $n$ from some binary field, and integer `t` and an integer `b` and outputs a generator polynomial of least possible degree having $a^b, a^{b+1}, \ldots, a^{b+2t-1}$ as roots.  **Hint:** You can use your implementation in Exercise 0 to find the minimal polynomial of powers of `a` with respect to $\mathbb{F}_2$.  The generator polynomial is a product of such minimal polynomials, but each such minimal polynomial needs to be included only once.  To see whether the minimal polynomial for $a^i$ has already been included in a candidate $g(x)$, simply evaluate $g(a^i)$ to see if $a^i$ is a zero.  Test your implementation by running the following cell.
   
2. Copy your function `grs_decoder` and modify it to create a function `bch_decoder(y::AbstractVector{GF2},a::Gf2,t::Int,b::Int)` that returns a binary error pattern vector corresponding to the received vector `y` for the code with generator polynomial given by `bch_generator(a,t,b)`.  **Hint:** the binary BCH code is a subcode of the RS code with a generator polynomial having $2t$ consecutive powers of `a` as zeros, starting at $a^b$.  There is no need to invoke the Forney formula, since there is only one possible error value.  The code locators and parity-check column multipliers can be computed from `a` and `b`.

In [ ]:
function bch_generator(a::Gf2,t::Int,b::Int)
    # fill in the rest
end

In [ ]:
α = gfprimitive(4)
bch_generator(α,2,1)  # we expect minpoly(alpha)*minpoly(alpha^3)

In [ ]:
minpoly(α) * minpoly(α^3)

In [ ]:
function bch_decoder(y::AbstractVector{GF2},a::Gf2,t::Int,b::Int)
    # fill in the rest, copying from grs_decoder
end

In [ ]:
α = gfprimitive(4)
t = 2
b = 1
g = bch_generator(α,t,b)
n = gforder(α)
k = n-degree(g)
println("Decoding the ($(n),$(k)) binary cyclic BCH code with generator polynomial $(g).")
TRIALS = 10000  # let's do this number of trials with errors in t random positions
FAILURES = 0
ERRORS = 0
for trial in 1:TRIALS
    u = GF2.(rand(0:1,k)) # generate a random k-bit message
    v = coeffs(Polynomial(u)*g)  # encode by multiplying by the generator polynomial
    while length(v) < n  # it could happen that we get a low degree, so pad with zeros at the top
        push!(v,zero(GF2))
    end
    y = copy(v)  # uncorrupted received word
    for j in 1:t
        y[rand(1:n)] += GF2(1)  # bit error in a random position
    end
    e = bch_decoder(y,α,t,b)
    if length(e) == 0
        FAILURES += 1
    else
        y -= e
        if (y != v)
            ERRORS += 1
        end
    end
end
println("After $(TRIALS) trials, $(ERRORS) errors and $(FAILURES) failures were encountered.")


---

This completes Numerical Exercise 2. Following the same directions as in Exercise 0, convert to html, and then print to PDF to create a file that can be uploaded on Quercus on or before the due date.